# RAG-based Indian Legal AI Assistant:
 Uses SentenceTransformer embeddings and FAISS for semantic retrieval over Indian legal cases, enabling Retrieval-Augmented Generation (RAG) for legal query answering.

## Delete Previous Records

In [ ]:
# !rm -r legal_ai_deployment
print("Removed")

Removed


## Unzip data(Mounted from drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/cjpe.zip' -d '/content/CJPE'
print("Unzipped")

Archive:  /content/drive/MyDrive/cjpe.zip
  inflating: /content/CJPE/il-tur-multi_train-00000-of-00002.arrow  
  inflating: /content/CJPE/il-tur-multi_train-00001-of-00002.arrow  
  inflating: /content/CJPE/il-tur-single_dev.arrow  
  inflating: /content/CJPE/il-tur-single_train.arrow  
  inflating: /content/CJPE/il-tur-test.arrow  
  inflating: /content/CJPE/dataset_info.json  
  inflating: /content/CJPE/il-tur-expert.arrow  
  inflating: /content/CJPE/il-tur-multi_dev.arrow  
Unzipped


## Installing Packages

In [ ]:
!pip install datasets
!pip install transformers
!pip install torch
!pip install numpy
!pip install sentence-transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

## Import Libraries

In [ ]:
import os
import json
import numpy as np
from datasets import Dataset, concatenate_datasets
import pyarrow as pa
import torch
from sentence_transformers import SentenceTransformer
import faiss
import re
from datetime import datetime
import pickle

## Load CJPE dataset

In [ ]:
print("Loading dataset")

try:
  dataset_path = "/content/CJPE"

  # load all splits
  cjpe_dataset = {
      'expert': Dataset.from_file(f"{dataset_path}/il-tur-expert.arrow"),
      'single_train': Dataset.from_file(f"{dataset_path}/il-tur-single_train.arrow"),
      'single_dev': Dataset.from_file(f"{dataset_path}/il-tur-single_dev.arrow"),
      'multi_train': concatenate_datasets([
          Dataset.from_file(f"{dataset_path}/il-tur-multi_train-00000-of-00002.arrow"),
          Dataset.from_file(f"{dataset_path}/il-tur-multi_train-00001-of-00002.arrow")
      ]),
      'multi_dev': Dataset.from_file(f"{dataset_path}/il-tur-multi_dev.arrow"),
      'test': Dataset.from_file(f"{dataset_path}/il-tur-test.arrow")
  }

  # combine splits
  all_cases = []
  for i in cjpe_dataset.values():
    all_cases.extend(i)

  print(f" Loaded {len(all_cases): ,} cases")
except Exception as e:
  print(f"Error : {e}")

Loading dataset


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


 Loaded  42,465 cases


## Build simple Knowledge Base

In [ ]:
print("Simple Knowledge Base")

legal_knowledge = {
    'case_summaries' : []
}

for i, case in enumerate(all_cases):
  if i % 10000 == 0:
    print(f" Processed {i}/{len(all_cases)} cases")

  legal_knowledge['case_summaries'].append({
      'id': case.get('id', f'case_{i}'),
      'text' : case.get('text', ''),
      'decision': 'ACCEPTED' if case.get('label') == 1 else 'REJECTED',
      'expert_1' : case.get('expert_1'),
      'expert_2' : case.get('expert_2'),
      'expert_3' : case.get('expert_3'),
      'expert_4' : case.get('expert_4'),
      'expert_5' : case.get('expert_5')
  })


print(f"Knowledge base ready with {len(legal_knowledge['case_summaries']):,} cases")

Simple Knowledge Base
 Processed 0/42465 cases
 Processed 10000/42465 cases
 Processed 20000/42465 cases
 Processed 30000/42465 cases
 Processed 40000/42465 cases
Knowledge base ready with 42,465 cases


## AI Assistant(Class Definition)

In [ ]:
class IndianLegalAssistant:
  def __init__(self, knowledge_base):
    self.knowledge_base = knowledge_base
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device being used : {self.device}")

    # Initialise model
    print("Loading model")
    # self.sentence_model = SentenceTransformer('all-mpnet-base-v2')
    # using a smaller model for ec2 deployment
    self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

    # create embeddings for each case(fast retrievals)
    self.prepare_case_embeddings()

    # convo patterns(Used to identify the user's intent)
    self.legal_patterns = {
        'case_inquiry': ['case', 'matter', 'legal issue', 'problem', 'dispute'],
        'procedure': ['procedure', 'process', 'how to', 'steps', 'file', 'appeal'],
        'rights': ['rights', 'fundamental rights', 'legal rights', 'constitutional'],
        'acts_laws': ['act', 'law', 'section', 'article', 'provision'],
        'court_info': ['court', 'judge', 'jurisdiction', 'venue', 'hearing'],
        'greetings' : ['hello', 'hi', 'hey', 'namaste', 'good morning', 'good afternoon', 'good evening']
    }

  # create the vector embeddings for all the cases
  def prepare_case_embeddings(self):
    print("Preparing case embeddings")

    case_texts = [case['text'] for case in self.knowledge_base['case_summaries']]
    # batch acc to t4
    self.case_embeddings = self.sentence_model.encode(case_texts, show_progress_bar=True, batch_size=128)

    # Make FAISS Index
    self.faiss_index = faiss.IndexFlatIP(self.case_embeddings.shape[1])
    self.faiss_index.add(self.case_embeddings.astype('float32'))

    print(f"Prepared {len(case_texts)} case embeddings")

  def find_similar_cases(self, query, top_k=3):
    # this will take the user's query and convert it into a vector using same model
    query_embedding = self.sentence_model.encode([query])

    # search similar cases
    scores, indices = self.faiss_index.search(query_embedding.astype('float32'), top_k)

    similar_cases = []
    # FAISS returns result in 2D array(first dimension has the query so indices[0])
    for i, idx in enumerate(indices[0]):
      similar_cases.append({
          'case' : self.knowledge_base['case_summaries'][idx],
          'similarity': scores[0][i]
      })
    return similar_cases

  # classify query intent(uses common convo to classify)
  def classify_intent(self, query):
    query_lower = query.lower()

    for intent, keywords in self.legal_patterns.items():
      if any(keyword in query_lower for keyword in keywords):
        return intent

    return 'general_legal'

  # Generate legal response
  def generate_legal_response(self, query):
    intent = self.classify_intent(query)
    similar_cases = self.find_similar_cases(query)

    response = {
        'intent': intent,
        'answer': '',
        'similar_cases': similar_cases,
        'legal_guidance': '',
        'relevant_laws': []
    }

    # Generate response based on intent
    if intent == 'greetings':
        response['answer'] = "Namaste! I'm your Indian Legal AI Assistant. I can help you understand Indian legal procedures, case precedents, and provide guidance on legal matters. How may I assist you today?"

    elif intent == 'case_inquiry':
        response['answer'] = self.generate_case_analysis_response(query, similar_cases)
        response['legal_guidance'] = self.extract_legal_guidance(similar_cases)

    elif intent == 'procedure':
        response['answer'] = self.generate_procedure_response(query, similar_cases)

    elif intent == 'rights':
        response['answer'] = self.generate_rights_response(query, similar_cases)

    elif intent == 'court_info':
        response['answer'] = self.generate_court_info_response(query, similar_cases)

    else:
        response['answer'] = self.generate_general_response(query, similar_cases)

    return response


  def generate_case_analysis_response(self, query, similar_cases):
    if not similar_cases:
      return "I don't have enough information about this specific case. Could you provide more details?"

    top_case = similar_cases[0]['case']
    similarity_score = similar_cases[0].get('similarity', 0)
    response = f"""
    📋 **Case Analysis:**
    Based on similar cases in Indian courts, here's what I found:
    **Similar Case:** {top_case.get('id', 'N/A')}
    **Decision:** {top_case.get('decision', 'UNKNOWN')}
    **Similarity Score:** {similarity_score:.3f}

    **Case Text:**
    {top_case.get('text', '')[:1000]}...
    [Full length: {len(top_case.get('text', ''))} characters]
    """

    # Add expert insights if available
    if top_case.get('expert_1') is not None:
        response += f"\n\n**Expert Annotations Available:**"
        for expert_num in range(1, 6):  # Show all experts
            expert_data = top_case.get(f'expert_{expert_num}')
            if expert_data and isinstance(expert_data, dict) and 'rank1' in expert_data:
                if expert_data['rank1']:
                    response += f"\n   Expert {expert_num}: {expert_data['rank1'][0][:300]}..."

    return response

  def generate_procedure_response(self, query, similar_cases):
      return """
      ⚖️ **Legal Procedure Guidance:**
      Based on Indian legal procedures and case precedents:

      **General Steps:**
      1. **Case Assessment** - Evaluate the legal merits of your case
      2. **Documentation** - Gather all relevant documents and evidence
      3. **Legal Consultation** - Consult with a qualified lawyer
      4. **Court Filing** - File appropriate petition/application in competent court
      5. **Service of Notice** - Serve notice to opposite parties
      6. **Court Proceedings** - Attend hearings and follow court directions
      7. **Judgment** - Await court's decision

      **Important Notes:**
      - Always consult a practicing lawyer for specific legal advice
      - Time limitations apply for different types of cases
      - Proper documentation is crucial for legal proceedings
      """

  def generate_rights_response(self, query, similar_cases):
      return """
      🛡️ **Legal Rights Information:**
      **Fundamental Rights (Constitution of India):**
      - Right to Equality (Articles 14-18)
      - Right to Freedom (Articles 19-22)
      - Right against Exploitation (Articles 23-24)
      - Right to Freedom of Religion (Articles 25-28)
      - Cultural and Educational Rights (Articles 29-30)
      - Right to Constitutional Remedies (Article 32)

      **Legal Rights:**
      - Right to legal representation
      - Right to fair trial
      - Right to appeal
      - Right to bail (in bailable offenses)
      - Right to remain silent

      **Note:** This is general information. Specific rights may vary based on circumstances and applicable laws.
      """

  def generate_court_info_response(self, query, similar_cases):
      return """
      🏛️ **Indian Court System:**
      **Supreme Court of India:**
      - Highest judicial authority
      - Constitutional and appellate jurisdiction
      - Located in New Delhi

      **High Courts:**
      - State-level highest courts
      - Original and appellate jurisdiction
      - 25 High Courts across India

      **District Courts:**
      - District-level courts
      - Trial courts for most cases
      - Civil and criminal jurisdiction

      **Specialized Courts:**
      - Family Courts, Consumer Courts, Labour Courts, Commercial Courts

      **Jurisdiction:** File cases in the appropriate court based on case type, value, and geographical location.
      """

  def generate_general_response(self, query, similar_cases):
      if similar_cases:
          top_case = similar_cases[0]['case']
          return f"""
          ⚖️ **Legal Information:**
          Based on your query and similar cases in our database:

          **Related Case Decision:** {top_case['decision']}

          **General Legal Guidance:**
          - Every legal matter is unique and requires careful analysis
          - Indian legal system follows common law principles with statutory modifications
          - Always seek professional legal advice for specific matters
          - Document everything related to your legal issue
          - Be aware of limitation periods for different types of legal actions

          **Disclaimer:** This is general information only and not specific legal advice.
          """
      else:
          return """
          ⚖️ I'd be happy to help with your legal query! Could you provide more specific details about:
          - The type of legal issue you're facing
          - Relevant laws or sections you're concerned about
          - The jurisdiction or court you're dealing with
          - Any specific procedural questions you have

          This will help me provide more targeted assistance based on Indian legal precedents and procedures.
          """


  def extract_legal_guidance(self, similar_cases):
      if not similar_cases:
          return "Consult with a qualified lawyer for specific legal advice."

      guidance = []
      for case in similar_cases[:2]:
          case_info = case['case']
          if case_info['decision'] == 'ACCEPTED':
              guidance.append("Similar cases have been decided favorably")
          else:
              guidance.append("Similar cases faced challenges - careful preparation needed")

      return '; '.join(guidance)


## Initialise Assistant

In [ ]:
print("Initialising Assistant")
legal_assistant = IndianLegalAssistant(legal_knowledge)
print("done")

Initialising Assistant
Device being used : cuda
Loading model
Preparing case embeddings


Batches:   0%|          | 0/332 [00:00<?, ?it/s]

Prepared 42465 case embeddings
done


## Save Model

In [ ]:
def save_model_files():
  # Create directory
  os.makedirs('legal_ai_deployment', exist_ok=True)
  # save model
  print("Saving model")
  legal_assistant.sentence_model.save('legal_ai_deployment/sentence_model')

  # save faiss
  print("Saving faiss")
  faiss.write_index(legal_assistant.faiss_index, 'legal_ai_deployment/legal_cases.faiss')

  # save case embeddings
  print("Saving case embeddings")
  np.save('legal_ai_deployment/case_embeddings.npy', legal_assistant.case_embeddings)

  # saving knowledge base
  print("Saving Knowledge Base")
  with open('legal_ai_deployment/knowledge_base.pkl', 'wb') as f:
    pickle.dump(legal_knowledge, f)

  print("Done")

  # save legal patterns and metadata
  print("Saving legal patterns")
  metadata = {
      'legal_patterns': legal_assistant.legal_patterns,
      'model_info': {
          # 'sentence_model': 'all-mpnet-base-v2',
          'sentence_model': 'all-MiniLM-L6-v2',
          'embedding_dim': legal_assistant.case_embeddings.shape[1],
          'total_cases': len(legal_knowledge['case_summaries']),
          'faiss_index_type': 'IndexFlatIP'
      },
      'created_at': datetime.now().isoformat()
  }
  with open('legal_ai_deployment/metadata.json', 'w') as f:
      json.dump(metadata, f, indent=2)

## Execute Model saving

In [ ]:
save_model_files()

Saving model
Saving faiss
Saving case embeddings
Saving Knowledge Base
Done
Saving legal patterns


## Chat Interface

In [ ]:
def chat_with_legal_assistant():
  print("\nAI ASSISTANT CHAT (type 'quit' to exit)")
  while True:
    try:
      user_query = input("You: ").strip()
      if user_query.lower() in ['quit', 'exit', 'bye']:
        print("Goodbye!")
        break
      if user_query.lower() == 'help':
        print("Ask about legal procedures, rights, courts, or cases. Type 'quit' to exit.")
        continue
      if not user_query:
        print("Please enter a legal question.")
        continue
      response = legal_assistant.generate_legal_response(user_query)
      print(f"\nAI: {response['answer']}")
      if response['legal_guidance']:
        print(f"Guidance: {response['legal_guidance']}")
    except KeyboardInterrupt:
      print("\nGoodbye!")
      break
    except Exception as e:
      print(f"Error: {e}")

## Test the Assistant

In [ ]:
print("\nTesting Indian Legal AI Assistant...")
test_queries = [
    "Hello, I need help with a legal matter",
    "How do I file an appeal in High Court?",
    "What are my fundamental rights?",
    "I have a property dispute with my neighbor",
    "Which court should I approach for a contract dispute?"
]
for i, query in enumerate(test_queries, 1):
  response = legal_assistant.generate_legal_response(query)
  print(f"Test {i}: {query}\n  AI: {response['answer'][:100]}...\n  Intent: {response['intent']}")


Testing Indian Legal AI Assistant...
Test 1: Hello, I need help with a legal matter
  AI: 
    📋 **Case Analysis:**
    Based on similar cases in Indian courts, here's what I found:
    **Si...
  Intent: case_inquiry
Test 2: How do I file an appeal in High Court?
  AI: 
      ⚖️ **Legal Procedure Guidance:**
      Based on Indian legal procedures and case precedents:
...
  Intent: procedure
Test 3: What are my fundamental rights?
  AI: 
      🛡️ **Legal Rights Information:**
      **Fundamental Rights (Constitution of India):**
      ...
  Intent: rights
Test 4: I have a property dispute with my neighbor
  AI: 
    📋 **Case Analysis:**
    Based on similar cases in Indian courts, here's what I found:
    **Si...
  Intent: case_inquiry
Test 5: Which court should I approach for a contract dispute?
  AI: 
    📋 **Case Analysis:**
    Based on similar cases in Indian courts, here's what I found:
    **Si...
  Intent: case_inquiry


## Custom input

In [ ]:
print("\nCustom Query Mode: AI Assistant (type 'quit' to exit)")
while True:
  user_query = input("You: ").strip()
  if user_query.lower() in ['quit', 'exit', 'bye']:
    print("Goodbye!")
    break
  if not user_query:
    print("Please enter a legal question.")
    continue
  response = legal_assistant.generate_legal_response(user_query)
  print(f"\nAI: {response['answer']}")
  if response['legal_guidance']:
    print(f"Guidance: {response['legal_guidance']}")
  print(f"Intent: {response['intent']}")


Custom Query Mode: AI Assistant (type 'quit' to exit)
You: i had a fight

AI: 
          ⚖️ **Legal Information:**
          Based on your query and similar cases in our database:

          **Related Case Decision:** ACCEPTED

          **General Legal Guidance:**
          - Every legal matter is unique and requires careful analysis
          - Indian legal system follows common law principles with statutory modifications
          - Always seek professional legal advice for specific matters
          - Document everything related to your legal issue
          - Be aware of limitation periods for different types of legal actions

          **Disclaimer:** This is general information only and not specific legal advice.
          
Intent: general_legal
You: quit
Goodbye!


## Debug

In [ ]:
print("\nDEBUG MODE: Inspect Embeddings and Retrieval (type 'quit' to exit)")
while True:
    debug_query = input("Debug Query: ").strip()
    if debug_query.lower() in ['quit', 'exit', 'bye']:
        print("Exiting debug mode.")
        break
    if not debug_query:
        print("Please enter a query.")
        continue
    query_embedding = legal_assistant.sentence_model.encode([debug_query])
    print(f"Query embedding shape: {query_embedding.shape}, dtype: {query_embedding.dtype}")
    print(f"Case embeddings shape: {legal_assistant.case_embeddings.shape}, dtype: {legal_assistant.case_embeddings.dtype}")
    print(f"FAISS index is trained: {getattr(legal_assistant.faiss_index, 'is_trained', 'N/A')}")
    print(f"FAISS index ntotal: {legal_assistant.faiss_index.ntotal}")
    scores, indices = legal_assistant.faiss_index.search(query_embedding.astype('float32'), 3)
    print("Top 3 similar cases:")
    for i, idx in enumerate(indices[0]):
        case = legal_assistant.knowledge_base['case_summaries'][idx]
        print(f"  Rank {i+1}: ID={case['id']}, Score={scores[0][i]:.4f}")
        print(f"    Preview: {case['text'][:200]}...\n")


DEBUG MODE: Inspect Embeddings and Retrieval (type 'quit' to exit)
Debug Query: i had a fight
Query embedding shape: (1, 384), dtype: float32
Case embeddings shape: (42465, 384), dtype: float32
FAISS index is trained: True
FAISS index ntotal: 42465
Top 3 similar cases:
  Rank 1: ID=1998_129, Score=0.2337
    Preview: Thomas, J. In this case of uxoricide the husband was found guilty by the Sessions Court but the High Court found him number guilty and acquitted him. That judgment of the High Court is in challenge in...

  Rank 2: ID=1975_219, Score=0.2258
    Preview:  N. Bhagwati, J. This appeal, by special leave, arises out of an internecine fight between two brothers and their respective families. The deceased Kana and Birdha were brothers and they had fields ad...

  Rank 3: ID=1987_480, Score=0.2118
    Preview: CIVIL APPELLATE JURISDICTION Civil Appeal Nos. 2780-81 of 1982 From the Judgment and Order dated 11.6. 1982 of the Karnataka High Court in Writ Petition No.3386 and 3387 of

## Experiment with Groq

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 11.9 MB/s eta 0:00:00


In [ ]:
from groq import Groq
groq_client = Groq(api_key="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")

def get_enhanced_legal_response(query):
    # Get relevant cases using existing RAG system
    response = legal_assistant.generate_legal_response(query)
    similar_cases = legal_assistant.find_similar_cases(query, top_k=3)

    # Prepare context from similar cases
    case_context = "\n\n".join([
        f"Case {i+1}:\nID: {case['case']['id']}\n"
        f"Decision: {case['case']['decision']}\n"
        f"Summary: {case['case']['text'][:500]}..."
        for i, case in enumerate(similar_cases)
    ])

    # Prepare prompt for Groq
    messages = [
        {"role": "system", "content": """You are an expert Indian legal assistant. Using the provided case precedents
        and legal context, provide detailed, professional legal guidance. Always include:
        1. Analysis of similar cases
        2. Potential legal approaches
        3. Practical next steps
        4. Important considerations
        5. Relevant legal principles
        Always end with a disclaimer about consulting a qualified lawyer."""},

        {"role": "user", "content": f"""Based on the following similar cases and the user's query,
        provide comprehensive legal guidance:

        User Query: {query}

        Relevant Case Precedents:
        {case_context}

        Please provide a detailed response in a clear, professional format."""}
    ]

    # Get enhanced response from Groq
    try:
        completion = groq_client.chat.completions.create(
            model="qwen-qwq-32b",
            messages=messages,
            temperature=0.6,
            max_completion_tokens=4096,
            top_p=0.95,
            stream=True,
            stop=None
        )

        # Collect the streamed response
        enhanced_response = ""
        print("\nEnhanced Legal Analysis:")
        for chunk in completion:
            content = chunk.choices[0].delta.content or ""
            enhanced_response += content
            print(content, end="")

        return enhanced_response

    except Exception as e:
        print(f"\nError generating enhanced response: {e}")
        return response['answer']  # Fallback to original response

print("\nEnhanced Legal Assistant Mode (type 'quit' to exit)")
while True:
    user_query = input("\nYou: ").strip()
    if user_query.lower() in ['quit', 'exit', 'bye']:
        print("Goodbye!")
        break
    if not user_query:
        print("Please enter a legal question.")
        continue

    enhanced_response = get_enhanced_legal_response(user_query)
    print(f"\nAI: {enhanced_response}")


Enhanced Legal Assistant Mode (type 'quit' to exit)

You: i had a fight

Enhanced Legal Analysis:

<think>
Okay, so I need to help the user who mentioned they had a fight. Let me look at the cases provided first. 

Case 1 (1998_129) is about uxoricide where the husband was acquitted. The fight between spouses led to the wife's death, but the court acquitted him. That's about criminal liability in domestic disputes.

Case 2 (1975_219) involves a family feud between brothers over a mud wall. The Supreme Court here might have dealt with criminal charges from the fight, but the summary isn't detailed. It's about property disputes leading to violence.

Case 3 (1987_480) was rejected, but it's a civil case about Bangalore's beautification. Not directly related to fights, so maybe not relevant here.

The user's query is just "I had a fight." They might be facing legal issues from a physical altercation. I need to outline possible legal approaches based on Indian law. 

First, analyze the nat

## Zip/Download Model

In [ ]:
!zip -r legal_ai_deployment.zip legal_ai_deployment/


  adding: legal_ai_deployment/ (stored 0%)
  adding: legal_ai_deployment/legal_cases.faiss (deflated 8%)
  adding: legal_ai_deployment/knowledge_base.pkl (deflated 72%)
  adding: legal_ai_deployment/case_embeddings.npy (deflated 8%)
  adding: legal_ai_deployment/sentence_model/ (stored 0%)
  adding: legal_ai_deployment/sentence_model/model.safetensors (deflated 9%)
  adding: legal_ai_deployment/sentence_model/vocab.txt (deflated 53%)
  adding: legal_ai_deployment/sentence_model/sentence_bert_config.json (deflated 4%)
  adding: legal_ai_deployment/sentence_model/modules.json (deflated 62%)
  adding: legal_ai_deployment/sentence_model/tokenizer_config.json (deflated 73%)
  adding: legal_ai_deployment/sentence_model/config.json (deflated 48%)
  adding: legal_ai_deployment/sentence_model/1_Pooling/ (stored 0%)
  adding: legal_ai_deployment/sentence_model/1_Pooling/config.json (deflated 57%)
  adding: legal_ai_deployment/sentence_model/config_sentence_transformers.json (deflated 34%)
  addi

In [ ]:
from google.colab import files
files.download('legal_ai_deployment.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Case Research

In [ ]:
def research_similar_cases(query, top_k=5):
  similar_cases = legal_assistant.find_similar_cases(query,top_k=top_k)
  print(f"Results for the {query}")
  print(f"Found {len(similar_cases)} similar cases")

  for i, case_data in enumerate(similar_cases):
    case = case_data['case']
    similarity_score = case_data.get('similarity', 0)

    # case preview
    print(f"Case {i+1}:")
    print(f"ID: {case.get('id', 'N.A.')}")
    print(f"Decision: {case.get('decision','N.A')}")
    print(f"Similarity Score: {similarity_score:.4f}")
    print("\nFull Text")
    print(case.get('text', ''))

    # Print expert annotations if present
    # if expert1 exists then expert 6 also exists in the database
    if case.get('expert_1') is not None:
      print("\nExpert Annotations:")
      for expert_num in range(1, 6):
        expert_data = case.get(f'expert_{expert_num}')
        if expert_data and isinstance(expert_data, dict) and 'rank1' in expert_data:
          if expert_data['rank1']:
            print(f"Expert {expert_num}: {expert_data['rank1'][0]}")

print(f"Research Mode(type 'quit' to exit)")
while True:
  research_query = input("Research Query: ").strip()
  if research_query.lower() in ['quit', 'exit', 'bye']:
    print("Exiting research mode.")
    break
  if not research_query:
    print("Please enter a query.")
    continue

  num_cases = input("number of cases to retrieve(default: 3)").strip()
  try:
    num_cases = int(num_cases) if num_cases else 3
  except ValueError:
    num_cases = 3
    print("Invalid input. Using 3 as fallback")

  research_similar_cases(research_query, top_k=num_cases)

Research Mode(type 'quit' to exit)
Research Query: a guy stole money from my house
number of cases to retrieve(default: 3)5\
Invalid input. Using 3 as fallback
Results for the a guy stole money from my house
Found 3 similar cases
Case 1:
ID: 2011_496
Decision: ACCEPTED
Similarity Score: 0.3782

Full Text
 On the 8th February, 2001 at about 1245p.m., Sanjeev Kumar deceased came to the premises of M s Manoharlal Laxman Dass, Old Anaj Mandi, Narela and asked for payments due to him from the owner. The Head Muneem, Shyam Lal, asked P.W. 6 Anil Sharma, another Muneem, to go to the State Bank of Indore, Narela where they had an account to withdraw Rs. 4 lakhs so that the same companyld be paid off to Sanjiv Kumar. Anil Sharma, accordingly, accompanied by Sanjiv Sharma P.W. 10 left on a two wheeler for the State Bank of Indore and withdrew Rs. 4 lakhs and after putting the currency in a bag left for their own shop. As they were passing through Dahiya X-ray Wali Gali, a motor cycle came from t